In [7]:
import cv2
import torch
import os
import time
from PIL import Image
from datetime import datetime
from transformers import AutoImageProcessor, SwinForImageClassification

# Load the Hugging Face model and processor from local directory
processor = AutoImageProcessor.from_pretrained("./model")  # change to HF repo name if needed
model = SwinForImageClassification.from_pretrained("./model")
model.eval()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Get label mapping
id2label = model.config.id2label

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Create folder to save violations
os.makedirs("violations", exist_ok=True)

# Open webcam
cap = cv2.VideoCapture(0)

# Cooldown settings
cooldown = 5  # seconds
last_save_time = 0
violation_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

    for (x, y, w, h) in faces:
        # Extract and preprocess face
        face_img = rgb[y:y+h, x:x+w]
        if face_img.size == 0:
            continue

        pil_image = Image.fromarray(face_img).resize((224, 224))
        inputs = processor(images=pil_image, return_tensors="pt").to(device)

        # Model prediction
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            pred_idx = logits.argmax(-1).item()
            label = id2label[pred_idx]

        # Decide box color based on label
        if "without" in label.lower():
            color = (0, 0, 255)  # Red for violation
            if current_time - last_save_time > cooldown:
                # Save full frame as violation image
                violation_count += 1
                last_save_time = current_time
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"violations/violation_{timestamp}.jpg"
                cv2.imwrite(filename, frame)
                print(f"[!] Violation saved: {filename}")
        elif "with" in label.lower():
            color = (0, 255, 0)  # Green for masked
        else:
            color = (0, 255, 255)  # Yellow for unknown

        # Draw box and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, color, 2)

    # Show violation count
    cv2.putText(frame, f"Violations: {violation_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow("Mask Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()


[!] Violation saved: violations/violation_20250617_183655.jpg
[!] Violation saved: violations/violation_20250617_183700.jpg
[!] Violation saved: violations/violation_20250617_183705.jpg
[!] Violation saved: violations/violation_20250617_183710.jpg
[!] Violation saved: violations/violation_20250617_183716.jpg
